# Cherry Pick script with PyLabRobot for Hamilton

# questions 

Does lld work? 

# To do

- [X] Copy Hamilton 223 specs from lab to fit in PLR <br>
- [ ] Test pipeting in each position to ensure the sites and carriers are correctly located
- [ ] Write code for cherry pick <br>
- [ ] Simulate code on PLR simulator <br>
- [ ] Test code with water.  <br>
- [ ] Implement in cherry_pick git repo <br>
- [ ] Test with foil seal. Can i get it to override LLD? <br>
  - [ ] How do i switch off the LLD?
- [ ] Test if i can pearse the seals <br>
- [ ] Test if i can split into the three parts. <br>
- [ ] How do i move this code over to the Cherry picking folders and still have it work? 

# Fun to do

Test using the ISWAP to move plates or remove lids <br>


In [1]:
# await lh.stop()
# await vis.stop()

In [2]:
# !pip install pyusb

# Settup Robot

In [3]:
simulate = False
# simulate = True

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import ChatterBoxBackend
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.resources.hamilton import STARDeck
from pylabrobot.resources.trash import Trash
from pylabrobot.resources.carrier import Coordinate


if simulate:
  lh = LiquidHandler(backend=ChatterBoxBackend(), deck=STARDeck())
  vis = Visualizer(resource=lh)
else:
  backend = STAR()
  lh = LiquidHandler(backend=backend, deck=STARDeck())

await lh.setup()

if simulate:
  await vis.setup()

# labware resources:
from pylabrobot.resources import (
  PLT_CAR_L5AC, # plate carrier
  STF_L, # standard volume 300ul tips with filter
  TIP_50ul_w_filter_L, # 50ul with filter landscape position
  HT_L, # Rack with 96 1000ul High Volume Tip
  corning_96_wellplate_360ul_flat, # OT plate type
  Cos_96_FL # corning alternative to OT plate
)

## there are no tube runners in PLR yet.

trash = lh.deck.get_trash_area()

# Upload deck configuration for Star B223

In [4]:
# import deck specs for Hamilton in building 223 room 032
from .hamilton_star_B223 import CVS_Shaker_DTU, TIP_CAR_NTR_A00, MFX_CAR_DTU, RGT_CAR_3R_A01, RGT_CAR_5R_A00, COOLER

In [5]:
shaker_carrier = CVS_Shaker_DTU(name='my_shaker_0')
tip_car = TIP_CAR_NTR_A00(name='tip_carrier')
tip_car_mfx = MFX_CAR_DTU(name='tip_carrier_mfx')
plt_car1 = PLT_CAR_L5AC(name='PLT_CAR_L5AC_1')
plt_car2 = PLT_CAR_L5AC(name='PLT_CAR_L5AC_2')
reservar_carrier1 = RGT_CAR_3R_A01(name='reservar_1')
reservar_carrier2 = RGT_CAR_3R_A01(name='reservar_2')
reservar_carrier3 = RGT_CAR_3R_A01(name='reservar_3_R5')
cooler_carrier1 = COOLER(name='cooler_1')
cooler_carrier2 = COOLER(name='cooler_2')


# Add labware

In [6]:

tip_car[0] = TIP_50ul_w_filter_L(name='tips_01_50ul') # for TIP_CAR_480_A00
tip_car[1] = TIP_50ul_w_filter_L(name='tips_02_50ul') # for TIP_CAR_480_A00
tip_car[2] = TIP_50ul_w_filter_L(name='tips_03_50ul') # for TIP_CAR_480_A00

tip_car_mfx[0] = STF_L(name='tips_01_300ul') # for MFX
tip_car_mfx[1] = STF_L(name='tips_02_300ul') # for MFX
tip_car_mfx[2] = HT_L(name='tips_01_1000ul') # for MFX
tip_car_mfx[3] = HT_L(name='tips_02_1000ul') # for MFX
tip_car_mfx[4] = HT_L(name='tips_03_1000ul') # for MFX


## Test on cooler
cooler_carrier1[1] = Cos_96_FL(name='source_1')
cooler_carrier1[2] = Cos_96_FL(name='source_2')
cooler_carrier1[3] = Cos_96_FL(name='source_3')
cooler_carrier1[4] = Cos_96_FL(name='source_4')
cooler_carrier2[4] = Cos_96_FL(name='dist1')


# Place labware and Carriers

In [9]:
location_shaker = Coordinate(x=77.5, y=63.0, z=100.0)
lh.deck.assign_child_resource(shaker_carrier, rails=None, location=location_shaker, reassign=True)
lh.deck.assign_child_resource(tip_car, rails=7)
lh.deck.assign_child_resource(plt_car1, rails=27)
lh.deck.assign_child_resource(plt_car2, rails=33)
lh.deck.assign_child_resource(tip_car_mfx, rails=13, reassign=True)
lh.deck.assign_child_resource(reservar_carrier1, rails=19, reassign=True)
lh.deck.assign_child_resource(reservar_carrier2, rails=20, reassign=True)
lh.deck.assign_child_resource(reservar_carrier3, rails=21, reassign=True)
lh.deck.assign_child_resource(cooler_carrier1, rails=43, reassign=True)
lh.deck.assign_child_resource(cooler_carrier2, rails=49, reassign=True)

In [ ]:
tiprack50_1 = lh.get_resource("tips_01_50ul")
tiprack50_2 = lh.get_resource("tips_02_50ul")
tiprack50_3 = lh.get_resource("tips_03_50ul")

tiprack300_1 = lh.get_resource("tips_01_300ul")
tiprack300_2 = lh.get_resource("tips_02_300ul")
tiprack1000_1 = lh.get_resource("tips_01_1000ul")
tiprack1000_2 = lh.get_resource("tips_02_1000ul")
tiprack1000_3 = lh.get_resource("tips_03_1000ul")

source1 = lh.get_resource('source_1')
source2 = lh.get_resource('source_2')
source3 = lh.get_resource('source_3')
source4 = lh.get_resource('source_4')
dist1 = lh.get_resource('dist1')

# Simulation create liquids in plates and fill tips

In [10]:


if simulate:
  # define liquids
  plate_1_liquids = [[(None, 100)]]*96

  source1.set_well_liquids(plate_1_liquids)
  source2.set_well_liquids(plate_1_liquids)
  source3.set_well_liquids(plate_1_liquids)
  source4.set_well_liquids(plate_1_liquids)
  # plate_2.set_well_liquids(plate_2_liquids)

# Liquid Commands

In [26]:
# variable input
t_list = [['10', 'A1', '9', 'A1', '100', 'FSH0345O'], ['10', 'B1', '9', 'B1', '100', 'FSH0345O'], ['10', 'C1', '9', 'C1', '100', 'FSH0345O'], ['10', 'D1', '9', 'D1', '100', 'FSH0345O'], ['10', 'E1', '9', 'E1', '100', 'FSH0345O'], ['10', 'F1', '9', 'F1', '100', 'FSH0345O'], ['10', 'G1', '9', 'G1', '100', 'FSH0345O'], ['10', 'H1', '9', 'H1', '100', 'FSH0345O'], ['10', 'A2', '9', 'A2', '100', 'FSH0345O'], ['10', 'B2', '9', 'B2', '100', 'FSH0345O'], ['10', 'C2', '9', 'C2', '100', 'FSH0345O'], ['10', 'D2', '9', 'D2', '100', 'FSH0345O'], ['10', 'E2', '9', 'E2', '100', 'FSH0345O'], ['10', 'F2', '9', 'F2', '100', 'FSH0345O'], ['10', 'G2', '9', 'G2', '100', 'FSH0345O'], ['10', 'H2', '9', 'H2', '100', 'FSH0345O'], ['10', 'A3', '9', 'A3', '100', 'FSH0345O'], ['10', 'B3', '9', 'B3', '100', 'FSH0345O'], ['10', 'C3', '9', 'C3', '100', 'FSH0345O'], ['10', 'D3', '9', 'D3', '100', 'FSH0345O'], ['10', 'E3', '9', 'E3', '100', 'FSH0345O'], ['10', 'F3', '9', 'F3', '100', 'FSH0345O'], ['10', 'G3', '9', 'G3', '100', 'FSH0345O'], ['10', 'H3', '9', 'H3', '100', 'FSH0345O'], ['10', 'A4', '9', 'A4', '100', 'FSH0345O'], ['10', 'B4', '9', 'B4', '100', 'FSH0345O'], ['10', 'C4', '9', 'C4', '100', 'FSH0345O'], ['10', 'D4', '9', 'D4', '100', 'FSH0345O'], ['10', 'E4', '9', 'E4', '100', 'FSH0345O'], ['10', 'F4', '9', 'F4', '100', 'FSH0345O'], ['10', 'G4', '9', 'G4', '100', 'FSH0345O'], ['10', 'H4', '9', 'H4', '100', 'FSH0345O'], ['10', 'A5', '9', 'A5', '100', 'FSH0345O'], ['10', 'B5', '9', 'B5', '100', 'FSH0345O'], ['10', 'C5', '9', 'C5', '100', 'FSH0345O'], ['10', 'D5', '9', 'D5', '100', 'FSH0345O'], ['10', 'E5', '9', 'E5', '100', 'FSH0345O'], ['10', 'F5', '9', 'F5', '100', 'FSH0345O'], ['7', 'A1', '9', 'G5', '100', 'FSH034AP'], ['7', 'B1', '9', 'H5', '100', 'FSH034AP'], ['7', 'C1', '9', 'A6', '100', 'FSH034AP'], ['7', 'E1', '9', 'B6', '100', 'FSH034AP'], ['7', 'A2', '9', 'C6', '100', 'FSH034AP'], ['7', 'D2', '9', 'D6', '100', 'FSH034AP'], ['7', 'E2', '9', 'E6', '100', 'FSH034AP'], ['7', 'F2', '9', 'F6', '100', 'FSH034AP'], ['7', 'G2', '9', 'G6', '100', 'FSH034AP'], ['7', 'B3', '9', 'H6', '100', 'FSH034AP'], ['7', 'E3', '9', 'A7', '100', 'FSH034AP'], ['7', 'F3', '9', 'B7', '100', 'FSH034AP'], ['7', 'G3', '9', 'C7', '100', 'FSH034AP'], ['7', 'A4', '9', 'D7', '100', 'FSH034AP'], ['7', 'C4', '9', 'E7', '100', 'FSH034AP'], ['7', 'F4', '9', 'F7', '100', 'FSH034AP'], ['7', 'H4', '9', 'G7', '100', 'FSH034AP'], ['7', 'A5', '9', 'H7', '100', 'FSH034AP'], ['7', 'B5', '9', 'A8', '100', 'FSH034AP'], ['7', 'C5', '9', 'B8', '100', 'FSH034AP'], ['7', 'D5', '9', 'C8', '100', 'FSH034AP'], ['7', 'E5', '9', 'D8', '100', 'FSH034AP'], ['7', 'F5', '9', 'E8', '100', 'FSH034AP'], ['7', 'G5', '9', 'F8', '100', 'FSH034AP'], ['7', 'H5', '9', 'G8', '100', 'FSH034AP'], ['7', 'E6', '9', 'H8', '100', 'FSH034AP'], ['7', 'F6', '9', 'A9', '100', 'FSH034AP'], ['7', 'G6', '9', 'B9', '100', 'FSH034AP'], ['7', 'H6', '9', 'C9', '100', 'FSH034AP'], ['7', 'A7', '9', 'D9', '100', 'FSH034AP'], ['7', 'B7', '9', 'E9', '100', 'FSH034AP'], ['7', 'C7', '9', 'F9', '100', 'FSH034AP'], ['7', 'D7', '9', 'G9', '100', 'FSH034AP'], ['7', 'E7', '9', 'H9', '100', 'FSH034AP'], ['7', 'F7', '9', 'A10', '100', 'FSH034AP'], ['7', 'G7', '9', 'B10', '100', 'FSH034AP'], ['7', 'H7', '9', 'C10', '100', 'FSH034AP'], ['7', 'A8', '9', 'D10', '100', 'FSH034AP'], ['7', 'B8', '9', 'E10', '100', 'FSH034AP'], ['7', 'C8', '9', 'F10', '100', 'FSH034AP'], ['7', 'D8', '9', 'G10', '100', 'FSH034AP'], ['7', 'E8', '9', 'H10', '100', 'FSH034AP'], ['7', 'F8', '9', 'A11', '100', 'FSH034AP'], ['7', 'G8', '9', 'B11', '100', 'FSH034AP'], ['7', 'H8', '9', 'C11', '100', 'FSH034AP'], ['7', 'A9', '9', 'D11', '100', 'FSH034AP'], ['7', 'B9', '9', 'E11', '100', 'FSH034AP'], ['7', 'C9', '9', 'F11', '100', 'FSH034AP'], ['7', 'F9', '9', 'G11', '100', 'FSH034AP'], ['7', 'G9', '9', 'H11', '100', 'FSH034AP'], ['7', 'A10', '9', 'A12', '100', 'FSH034AP'], ['7', 'B10', '9', 'B12', '100', 'FSH034AP'], ['7', 'C10', '9', 'C12', '100', 'FSH034AP'], ['7', 'D10', '9', 'D12', '100', 'FSH034AP']]

In [15]:
# tip configuration in box:

tip_slot = ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']

# fixed input
control_input = {'source_plate': 'biorad_96_wellplate_200ul_pcr', 'dest_plate': 'usascientific_96_wellplate_2.4ml_deep', 'right_pipette': 'p300_single', 'left_pipette': 'p10_single'}

In [16]:
input_slots = {10: source1, 7: source2, 4: source3, 1: source4, 9: dist1}
def find_plate_name(input_slots, source_slot):
    for key, value in input_slots.items():
        if key == source_slot:
            return value
    return None  # Return None if no value found for the given key
test_input = 10
find_plate_name(input_slots, test_input)


Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))

In [17]:
## Testing with

count = 1

source_slot = t_list[count][0]
source_well = t_list[count][1]
dist_slot = t_list[count][2]
dist_well = t_list[count][3]
volume = float(t_list[0][4])
source_plate = find_plate_name(input_slots, int(source_slot))
print('source_plate', source_plate)
print('source_well', source_well)
print(t_list[count])

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'B1', '9', 'B1', '100', 'FSH0345O']


In [19]:
volumes = []
source_slots = []
source_wells = []
dist_slots = []
dist_wells = []
source_plates = []
dist_plates = []

for count in range(len(t_list)):
  source_slots.append(t_list[count][0])
  source_wells.append(t_list[count][1])
  dist_slots.append(t_list[count][2])
  dist_wells.append(t_list[count][3])
  volumes.append(float(t_list[0][4]))
  source_plates.append(find_plate_name(input_slots, int(source_slots[count])))
  dist_plates.append(find_plate_name(input_slots, int(dist_slots[count])))
  print('source_plate', source_plate)
  print('source_well', source_well)
  print(t_list[count])
  print()


source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'A1', '9', 'A1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'B1', '9', 'B1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'C1', '9', 'C1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'D1', '9', 'D1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'E1', '9', 'E1', '100', 'FSH0345O']

source_plate Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000))
source_well B1
['10', 'F1',

In [21]:

def all_elements_same(source_plates):
    ## test if a list of len=8 contains the same plate if yes aspirate all 8 at the same time if not then aspirate one at a time.
    # Check if the list is empty
    if not source_plates:
        return True
    # Compare all elements to the first element
    first_element = source_plates[0]
    return all(element == first_element for element in source_plates[1:])

In [125]:

## create a tip pick up nested list for doing all work in columns(ei use 8 channels)
multiples_of_8  = [0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96]
tip_pick_up_eight = []
for i in range(len(multiples_of_8)-1):
  if i == 0:
    start = multiples_of_8[i]
  else:
    start = multiples_of_8[i]

  end = multiples_of_8[i+1]
  tip_pick_up_eight.append([start, end])

print(tip_pick_up_eight)

[[0, 8], [8, 16], [16, 24], [24, 32], [32, 40], [40, 48], [48, 56], [56, 64], [64, 72], [72, 80], [80, 88], [88, 96]]


In [126]:
count = 4
start, end = tip_pick_up_eight[count][0] , tip_pick_up_eight[count][1]
print(start, end)

32 40


In [127]:
print(volumes[start:end])
print(source_slots[start:end])
print(source_wells[start:end])
print(dist_slots[start:end])
print(dist_wells[start:end])
print(source_plates[start:end])

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
['10', '10', '10', '10', '10', '10', '7', '7']
['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'A1', 'B1']
['9', '9', '9', '9', '9', '9', '9', '9']
['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5']
[Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_1, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_2, size_x=127.0, size_y=86.0, size_z=14.24, location=(000.000, 000.000, 000.000)), Plate(name=source_2, size_x=127.0, size_y=86.0, s

In [128]:
print(start - end)
print(tip_slot[start-1:end])
await lh.pick_up_tips(tiprack300_1[tip_slot[start:end]])

-8
['H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5']
Picking up tips [Pickup(resource=TipSpot(name=tips_01_tipspot_4_0, location=(043.200, 068.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspot_4_1, location=(043.200, 059.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspot_4_2, location=(043.200, 050.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Pickup(resource=TipSpot(name=tips_01_tipspo

Aspirating [Aspiration(resource=Well(name=source_2_well_0_1, location=(010.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].


In [131]:

if all_elements_same(source_plates[start:end]):
  source_plate = source_plates[0]
  await lh.aspirate(source_plate[source_wells[start:end]], vols=volumes[start:end])#, swap_speed=100, homogenization_speed=1000)
else:
  # aspirate one at a time
  for i in range(start, end):
    channel = i - start
    source_plate = source_plates[i]
    await lh.aspirate(source_plate[source_wells[i]], vols=volumes[i], use_channels=[channel])

Aspirating [Aspiration(resource=Well(name=source_1_well_4_0, location=(046.570, 071.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].
Aspirating [Aspiration(resource=Well(name=source_1_well_4_1, location=(046.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)])].
Aspirating [Aspiration(resource=Well(name=source_1_well_4_2, location=(046.570, 053.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTi

In [132]:
if all_elements_same(dist_plates[start:end]):
  dist1 = dist_plates[0]
  await lh.dispense(dist1[dist_wells[start:end]], vols=volumes[start:end])#, swap_speed=100, homogenization_speed=1000, mix_speed=1000)
else:
  # dispense one at a time
  for i in range(start, end):
    channel = i - start
    dist1 = dist_plates[i]
    await lh.dispense(dist1[dist_wells[i]], vols=volumes[i], use_channels=[channel])



Dispensing [Dispense(resource=Well(name=dist1_well_4_0, location=(046.570, 071.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)]), Dispense(resource=Well(name=dist1_well_4_1, location=(046.570, 062.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK), volume=100.0, flow_rate=None, liquid_height=None, blow_out_air_volume=None, liquids=[(None, 100.0)]), Dispense(resource=Well(name=dist1_well_4_2, location=(046.570, 053.070, 001.000), size_x=6.86, size_y=6.86, size_z=10.67, category=well), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maxi

In [133]:
await lh.drop_tips(tiprack300_1[tip_slot[start:end]])

Dropping tips [Drop(resource=TipSpot(name=tips_01_tipspot_4_0, location=(043.200, 068.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_1, location=(043.200, 059.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_2, location=(043.200, 050.300, -40.500), size_x=9.0, size_y=9.0, size_z=0, category=tip_spot), offset=None, tip=HamiltonTip(STANDARD_VOLUME, has_filter=False, maximal_volume=65, fitting_depth=8, total_tip_length=50.4, pickup_method=OUT_OF_RACK)), Drop(resource=TipSpot(name=tips_01_tipspot_4_3, location=(043.200, 041.300, -40.500), size_x=9.0, size_y=9.0,

In [ ]:
print(lh.deck.summary())

Rail     Resource                   Type                Coordinates (mm)
(-13) ├── trash_core96               Trash               (-232.100, 110.300, 189.000)
      │
(0)   ├── my_shaker_carrier          ShakerCarrier       (077.500, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(7)   ├── tip_carrier                TipCarrier          (235.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── tip01                  TipRack             (252.900, 241.800, 174.450)
      │   ├── tip02                  TipRack             (252.900, 337.800, 174.450)
      │   ├── <empty>
      │   ├── <empty>
      │
(13)  ├── my_mfx_carrier             MFXCarrier          (370.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(19)  ├── my_RGT_CAR_3R_A01_1        TubeCarrier         (505.000, 063.000, 100.000)
      │   ├──

# Shutdown

In [40]:
await lh.stop()